In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from webxplore import WebScraper
import time

In [ ]:
defaultSearchLink = 'http://www.google.com/search?q='
domainName = 'fool.com'
stockName = 'tesla'

dateStart = '20190101'
dateEnd = '20200101'

In [ ]:
finalSearchLink = defaultSearchLink + \
        f'site:{domainName}+{stockName}+after:{dateStart}+before:{dateEnd}&num=100'
finalSearch = f'site:{domainName} {stockName}'

In [ ]:
listLinks = []
i = 0

while True:
    time.sleep(3)
       
    resultPage = requests.get(f'{finalSearchLink}&start={100*i}')
    htmlParser = BeautifulSoup(resultPage.content)
    
    links = htmlParser.findAll("a")

    for link in htmlParser.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
        specifiedLink = re.split(":(?=http)",link["href"].replace("/url?q=",""))
        listLinks.append((''.join(specifiedLink)).split("&sa=U")[0])
    
    if len(listLinks) > 0:
        listLinks.pop(-1)

    blacklist = ['style','script']
    text_elements = [t for t in htmlParser.find_all(text=True) if 
                     t.parent.name not in blacklist]

    if 'repeat the search with the omitted results included' in text_elements:
        break
    i += 1


In [ ]:
from googleapiclient.discovery import build

my_api_key = ""
my_cse_id = ""

def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return [specificItem['link'] for specificItem in res['items']]

In [ ]:
listLinks = []
i = 0

while True:
    try:
        listLinks.extend(google_search(
            finalSearch, my_api_key, my_cse_id, num=10, start=10*i,
            sort=f'date:r:{dateStart}:{dateEnd}'))
        i += 1
    except Exception as exp:
        break

In [ ]:
relevantLinks = []
for individualLink in listLinks:
    time.sleep(2)
    
    try:
        webScraper = WebScraper.ScrapeWebsite(individualLink)
        websiteContent = webScraper.return_article()
    except Exception:
        continue

    if 'stock' in websiteContent.split():
        keyWords = ''
        relevantLinks.append(individualLink)